단어를 vector화 하는 표현 방법에 대해서 이해 했다면 이러한 표현방법에 대하여 코사인 유사도를 이용하여 문서의 유사도를 구하는것이 가능하다.

두 벡터(두 문서)가 같은 방향을 가지면 1 반대 방향을 가지면 -1
=> 코사인 유사도는 -1 ~ 1사이의 값을 가진다. 코사인유사도가 1에 가까워질수록 유사한 문장

In [1]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
    return dot(A, B) / (norm(A)*norm(B))

In [2]:
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


In [18]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

data= pd.read_csv("./archive/movies_metadata.csv", low_memory=False)
data.describe()


,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [9]:
data["overview"].isnull().sum()

954

In [19]:
data["overview"] = data["overview"].fillna('')
data["overview"].isnull().sum()

0

In [22]:
tfidf = TfidfVectorizer(stop_words="english")
tfidf_mat = tfidf.fit_transform(data["overview"])
print(tfidf_mat.shape)

(45466, 75827)


In [23]:
# 코사인 유사도
cosin_sim = linear_kernel(tfidf_mat, tfidf_mat)
print(cosin_sim.shape)

In [26]:
indices = pd.Series(data.index, index=data["title"]).drop_duplicates()
print(indices.head())


title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64
(45466, 45466)


In [48]:
def get_recommendations(title, cosine_sim):
    idx = indices[title]
    sim_score = sorted(list(enumerate(cosine_sim[idx])), key=lambda x : x[1], reverse=True)
    movie_idx = [i[0] for i in sim_score[1: 11]]
    print(movie_idx)
    
    return data["title"].iloc[movie_idx]


In [49]:
get_recommendations('The Dark Knight Rises', cosin_sim)

[12481, 150, 1328, 15511, 585, 21194, 9230, 18035, 19792, 3095]


12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object